***
# Creating SourceLists for SA113 (to create a mock-up MasterTable)
***

### Context and Purpose of this Notebook: 

This Notebook is to show the reader the steps we need to take in the Astro_WISE environment to be able to create SourceLists with the required parameters to do our position/flux analysis. This will give us more insight in how we can distinguish changes in the position that are due to proper motions of stars from those due to distortions or 'bad' astrometrical solutions. This is a useful tool to eventually be able QUALIFY and if UPDATE existing astrometrical solutions.
     

### Method: 
We choose SA 113 as our Standard Area. From all observations, let's call them RawScience frames, taken of this specific part of the sky, we will select the frames for detector `ESO_CCD_#90` in the `OCAM_r_SDSS` band and using only the `'STD,ZEROPOINT'` objects. These contain observations that last 75 seconds. 

### Results: 

Will be written based on outcome of below's analysis.

### Content:

* [Using the Astro-WISE pipelines to create astrometric solutions](#AWE)
    *  [From RawScienceFrame to ReducedScienceFrame](#raw)
    *  [From ReducedScienceFrame to AstrometricParameters](#red)
    *  [From AstrometricParameters to SourceLists](#ast)



In [1]:
from __future__ import print_function

import warnings
warnings.filterwarnings("ignore")
%run py/setup_awe

<a id = "AWE" AWE></a>
***
# Example of Astro-WISE pipelines to create astrometric solutions
***
These first three kernels are meant to be taken as an example for an image reducing pipeline. With the Astro_WISE objects, RawScienceFrame, ReducedScienceFrame, AstromParameters are we able to create Sourcelits that contain all the source extractions from the raw observations. 

We will use the basic building block that can be used in the awe propmt to give a clear overview of what steps are taken in the imaging pipeline(s).

The first step we need to be sure of when using the Astro-WISE system, is that we are querying in the right `project`. In our case this will be 'OMEGACAM@VST' since we run this example on a RawScienceFrame present in the Standard Area 113. We set the `privileges` to 3, meaning we are able to see the data from all Astro-WISE users present in the 'OMEGACAM@VST' project. 

In [2]:
context.set_project('OMEGACAM@VST')
context.set_privileges(3)

True

<br>

<a id = "SA113" SA113></a>
***
# Using Astro-WISE to create SourceLists for Standard Area 113
***

## Finding and accessing the data items from SA 113

We want to analyse and quality control the <b>SA113 monitoring field calibration</b> observations: astrometric model, astrometric reference catalog, astrometric solutions and derived astrometry of stars in SA113.

See [HOW-TO Use your Astro-WISE Context to set data scopes](http://doc.astro-wise.org/man_howto_context.html)

We start by making an inventory of all available raw exposures with(in):
* `OMEGACAM@VST` project
* Privilege level 3
* SA113 field, pointing RA= 325.375, DEC= 0.499
* Field of View of 1 squared degree
* DectectorID: `ESO_CCD_#90`
* Filter: `OCAM_r_SDSS`

In [67]:
context.set_project('OMEGACAM@VST')
context.set_privileges(3)
radec_SA113=[325.375,0.499]
ra_SA113,dec_SA113,dra_SA113,ddec_SA113=radec_SA113[0],radec_SA113[1],1.,1.

radec_SA107=[234.825,-0.263] 
ra_SA107,dec_SA107,dra_SA107,ddec_SA107=radec_SA107[0],radec_SA107[1],1.,1.

# radec_SA101=[149.112,-0.386]
# ra,dec,dra,ddec=radec_SA101[0],radec_SA101[1],1.,1.

In [68]:
qrawsfall_SA113=(RawScienceFrame.chip.name=='ESO_CCD_#90') & (RawScienceFrame.astrom.CRVAL1<ra_SA113+dra_SA113) & (RawScienceFrame.astrom.CRVAL1>ra_SA113-dra_SA113) & (RawScienceFrame.astrom.CRVAL2<dec_SA113+ddec_SA113) & (RawScienceFrame.astrom.CRVAL2>dec_SA113-ddec_SA113)
qrawsf_SA113= qrawsfall_SA113 & (RawScienceFrame.OBJECT=='STD,ZEROPOINT') & (RawScienceFrame.filter.name=='OCAM_r_SDSS') & (RawScienceFrame.EXPTIME==75.0) 
n_qrawsfall_pub_SA113=len(qrawsfall_SA113)
n_qrawsf_pub_SA113=len(qrawsf_SA113)

print("From all {} public RawScienceFrame's, we select {} based on our criteria".format(n_qrawsfall_pub_SA113, n_qrawsf_pub_SA113))

From all 4198 public RawScienceFrame's, we select 560 based on our criteria


In [69]:
qrawsfall_SA107=(RawScienceFrame.chip.name=='ESO_CCD_#90') & (RawScienceFrame.astrom.CRVAL1<ra_SA107+dra_SA107) & (RawScienceFrame.astrom.CRVAL1>ra_SA107-dra_SA107) & (RawScienceFrame.astrom.CRVAL2<dec_SA107+ddec_SA107) & (RawScienceFrame.astrom.CRVAL2>dec_SA107-ddec_SA107)
qrawsf_SA107= qrawsfall_SA107 & (RawScienceFrame.OBJECT=='STD,ZEROPOINT') & (RawScienceFrame.filter.name=='OCAM_r_SDSS') & (RawScienceFrame.EXPTIME==75.0) 
n_qrawsfall_pub_SA107=len(qrawsfall_SA107)
n_qrawsf_pub_SA107=len(qrawsf_SA107)

print("From all {} public RawScienceFrame's, we select {} based on our criteria".format(n_qrawsfall_pub_SA107, n_qrawsf_pub_SA107))

From all 4118 public RawScienceFrame's, we select 505 based on our criteria




### From RawSienceFrame to SourceList, step-by-step

Let's start with `simple exercise` to go from a RawScienceFrame --> ReducedScienceFrame --> AstromParams --> SourceList<br>
During this exercision, we constantly want to check whether there is already an existing ReducedScienceFrame or even AstromParams to not redo/re-run unnecessary pipelines.

In [ ]:
rawsf_filenames_SA113 = [rawframe.filename for rawframe in qrawsf_SA113]
print(len(rawsf_filenames_SA113))

In [ ]:
qredsf_SA113 = []
for filename in rawsf_filenames_SA113:
    if len((ReducedScienceFrame.raw.filename == filename)) > 0:
        qredsf_loop_SA113 = (ReducedScienceFrame.raw.filename == filename)
        qredsf_SA113.append(qredsf_loop_SA113)
    else:
        pass
n_qredsf_pub_SA113 = len(qredsf_SA113)
print("From all {} selected RawScienceFrame's, {} ReducedScienceFrame's were generated".format(n_qrawsf_pub_SA113, n_qredsf_pub_SA113))

In [7]:
rawsf_filenames_SA107 = [rawframe.filename for rawframe in qrawsf_SA107]
print(len(rawsf_filenames_SA107))

505


In [8]:
qredsf_SA107 = []
for filename in rawsf_filenames_SA107:
    if len((ReducedScienceFrame.raw.filename == filename)) > 0:
        qredsf_loop_SA107 = (ReducedScienceFrame.raw.filename == filename)
        qredsf_SA107.append(qredsf_loop_SA107)
    else:
        pass
n_qredsf_pub_SA107 = len(qredsf_SA107)
print("From all {} selected RawScienceFrame's, {} ReducedScienceFrame's were generated".format(n_qrawsf_pub_SA107, n_qredsf_pub_SA107))

From all 505 selected RawScienceFrame's, 505 ReducedScienceFrame's were generated


``` bash
# for SA107
From all 4118 public RawScienceFrames, we select 505 based on our criteria

Sci-EHELMICH-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-55994.6913563-12e5e68216bab5f0657a48030f0abb590fbb28c9.fits
Sci-EHELMICH-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-56548.3788899-4562c05d8d1e2745192ba5c95f2ed89217aeaf7a.fits
From all 505 selected RawScienceFrames, 2 ReducedScienceFrames were generated

```

In [9]:
apsf_filenames_SA113 = [redframe[0].filename for redframe in qredsf_SA113]
qapsf_SA113 = []
for filename in apsf_filenames_SA113:
    if len((AstrometricParameters.reduced.filename == filename)) > 0:
        qapsf_loop_SA113 = (AstrometricParameters.reduced.filename == filename)
#         print(qapsf_loop_SA113[0].creation_date, qapsf_loop_SA113[0].is_valid)
        qapsf_SA113.append(qapsf_loop_SA113)
n_qapsf_pub_SA113 = len(qapsf_SA113)
print("From all {} selected ReducedScienceFrame's, {} AstrometricParameters were generated".format(n_qredsf_pub_SA113, n_qapsf_pub_SA113))


2012-03-08 16:36:43 1
2013-09-13 09:06:44 1
From all 2 selected ReducedScienceFrame's, 2 AstrometricParameters were generated


In [9]:
apsf_filenames_SA107 = [redframe[0].filename for redframe in qredsf_SA107]
qapsf_SA107 = []
for filename in apsf_filenames_SA107:
    if len((AstrometricParameters.reduced.filename == filename)) > 0:
        qapsf_loop_SA107 = (AstrometricParameters.reduced.filename == filename)
#         print(qapsf_loop_SA107[0].creation_date, qapsf_loop_SA113[0].is_valid)
        qapsf_SA107.append(qapsf_loop_SA107)
n_qapsf_pub_SA107 = len(qapsf_SA107)
print("From all {} selected ReducedScienceFrame's, {} AstrometricParameters were generated".format(n_qredsf_pub_SA107, n_qapsf_pub_SA107))


From all 505 selected ReducedScienceFrame's, 505 AstrometricParameters were generated


Just as seen from our query 

``` python

# for SA113
> qap= (AstrometricParameters.chip.name=='ESO_CCD_#90') & (AstrometricParameters.filter.name=='OCAM_r_SDSS') & (AstrometricParameters.CRVAL1<ra+dra) & (AstrometricParameters.CRVAL1>ra-dra)  & (AstrometricParameters.CRVAL2<dec+ddec) & (AstrometricParameters.CRVAL2>dec-ddec)
> len(qap)
69

# for SA107
qap= (AstrometricParameters.chip.name=='ESO_CCD_#90') & (AstrometricParameters.filter.name=='OCAM_r_SDSS') & (AstrometricParameters.CRVAL1<ra+dra) & (AstrometricParameters.CRVAL1>ra-dra)  & (AstrometricParameters.CRVAL2<dec+ddec) & (AstrometricParameters.CRVAL2>dec-ddec)
> len(qap)
2
```

Only for a few RawScienceFrame's there is a AstrometricParameters object from which we can create a SourceList. Let's try to produce a ReducedScienceFrame, AstometricParameters and SourceList

## Example: One route to go from a RawScienceFrame to a SourceList 

In [ ]:
def check_existence_reduced(frame, CLASS = 'ReducedScienceFrame'):
    check = (ReducedScienceFrame.raw.filename == frame.filename)
    if len(check) == 0:
        print("There is no existing {} for the input frame with filename {}".format(CLASS, frame.filename))
    else:
        print("There is a {} for the input frame with filename {}".format(CLASS, frame.filename))

rawf = qrawsf[0]
rawf_filename = rawf.filename
check_existence_reduced(rawf)


<a id = "raw" SA113></a>

### 1) Using the [HOW-TO Create a ReducedScienceFrame](http://doc.astro-wise.org/man_howto_reduce.html) to obtain ReducedScienceFrame

In [ ]:
#http://doc.astro-wise.org/man_howto_reduce.html from astro.main.MasterFlatFrame import MasterFlatFrame
from astro.main.ColdPixelMap import ColdPixelMap
from astro.main.HotPixelMap import HotPixelMap
from astro.main.BiasFrame import BiasFrame


run = 'off'

if run == 'on':
    raw  = (RawScienceFrame.filename == rawf_filename)[0]
    hot  = HotPixelMap.select_for_raw(raw)
    cold = ColdPixelMap.select_for_raw(raw)
    flat = MasterFlatFrame.select_for_raw(raw)
    bias = BiasFrame.select_for_raw(raw)

    reduced = ReducedScienceFrame()
    reduced.raw  = raw
    reduced.hot  = hot
    reduced.cold = cold
    reduced.bias = bias
    reduced.flat = flat
    reduced.raw.retrieve()
    reduced.hot.retrieve()
    reduced.cold.retrieve()
    reduced.bias.retrieve()
    reduced.flat.retrieve()
    reduced.set_filename()

    reduced.make()
    reduced.store()
    reduced.weight.store()
    reduced.commit()



In [3]:
run = 'off'

if run == 'on':
    dpu.run('Reduce', i='OMEGACAM', raw_filenames=[dpu_framesrun], C=1, dpu_time=1799, send_code=False)
    dpu.run('Reduce', i='OMEGACAM', raw_filenames=[rawsf_filenames], C=1, dpu_time=1799, send_code=False)

qredsf = []
for idx, filename in enumerate(rawsf_filenames):
    dpu.run('Reduce', i='OMEGACAM', raw_filenames=[filename], C=0, dpu_time=1799, send_code=False)
    print("{}/{}".format(idx+1, len(redsf_filenames)))
n_qredsf_pub = len(qredsf)

<a id = "red" SA113></a>

### 2)  Using the [HOW-TO Derive astrometry](http://doc.astro-wise.org/man_howto_astrom.html#howtoastrom) to obtain AstrometricParameters

To go from a ReducesScienceFrame to AstrometricParameters, Astro-WISE users can use either one of the two commands:
```python
dpu.run('Astrometry', i='OMEGACAM', red_filenames=[reduced.filename], C=0)
```
or
```python
task = AstrometricParametersTask(red_filenames=[reduced.filename], commit=0)
task.execute()
```
as a fast tool to determine the astrometrical solutions. In the kernel below we show the steps you have to make by again using basic building blocks. 

Be aware, a lot of code is 'hiding' behind these methods. In any case, for all the three different approches, the system goes though the following steps to derive the solutions:

1. Approximation of the affine transformation parameters by preastrometric pairing (preastrom)
* Application of the affine transformation to the source catalogue (aplastrom)
* Association of source and astrometric reference catalogue (associate)
* Filter assaciations with large association distance
* Building of an PAIR_X catalogue based on the filtered associations (make_ssc)
* Derivation of the full astrometric solution (astrom)
* Conversion of the astrometric solution parameters to PV parameters (make_distort)

In [ ]:
def check_existence_astromp(frame, CLASS = 'AstrometricParameters'):
    check = (AstrometricParameters.reduced.filename == frame.filename)
    if len(check) == 0:
        print("There is no existing {} for the input frame with filename {}".format(CLASS, frame.filename))

for i in redsf_filenames:
    try:
        redf = (ReducedScienceFrame.raw.filename == i)
        redf_filename = redf[0].filename
        print(redf_filename)
    except IndexError as err:
        pass
    
redf_SA113 = (ReducedScienceFrame.raw.filename == rawf_filename_SA113)[0]
redf_filename_SA113 = redf.filename
check_existence_astromp(redf_SA113)


# ### EXAMPLE: GET FIRST REDUCED FRAME BY SEARCHING FOR FIRST RAW FRAME
# redf = (ReducedScienceFrame.raw.filename == 'OMEGACAM.2014-08-05T02:22:19.268_14.fits')[0]
# redf_filename = redf.filename
# print (redf_filename)

In [5]:
rawsf_filenames_SA113 = [rawframe.filename for rawframe in qrawsf_SA113]
redsf_filenames_SA113 = [(ReducedScienceFrame.raw.filename == rawf_filename)[0].filename for rawf_filename in rawsf_filenames_SA113]

aps_SA113 = [(AstrometricParameters.reduced.filename == redf_filename)[0] for redf_filename in redsf_filenames_SA113]

run = 'off'

if run == 'on':
    dpu.run('Astrometry', instrument='OMEGACAM', red_filenames=redsf_filenames_SA113, C=1, dpu_time=1799, send_code=False)

for slid in slids:
    SL = (SourceList.SLID == slid)[0].frame.filename

if run == 'on':
    task = AstrometricParametersTask(red_filenames=['Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58969.1264780-5e8f0d4868869ea54b6a5a82b8ca2d944c9e94a0.fits'], i='OMEGACAM')
    task.execute()

In [ ]:
# rawsf_filenames_SA107 = [rawframe.filename for rawframe in qrawsf_SA107]
# redsf_filenames_SA107 = [(ReducedScienceFrame.raw.filename == rawf_filename)[0].filename for rawf_filename in rawsf_filenames_SA107]
#print(redsf_filenames_SA113)

aps_SA107 = [(AstrometricParameters.reduced.filename == redf_filename)[0] for redf_filename in redsf_filenames_SA107]

# dpu.run('Astrometry', instrument='OMEGACAM', process_params.REFCAT='USNO-B1.0', red_filenames=redsf_filenames_SA107, C=1, dpu_time=1799, send_code=False)

for slid in slids:
    SL = (SourceList.SLID == slid)[0].frame.filename

# task = AstrometricParametersTask(red_filenames=['Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58969.1264780-5e8f0d4868869ea54b6a5a82b8ca2d944c9e94a0.fits'], i='OMEGACAM')
# task.execute()

<br>

#### AstrometricParameters with USNO-B1.0 as REFCAT

In [149]:
qrawsfall_SA113=(RawScienceFrame.chip.name=='ESO_CCD_#90') & (RawScienceFrame.astrom.CRVAL1<ra_SA113+dra_SA113) & (RawScienceFrame.astrom.CRVAL1>ra_SA113-dra_SA113) & (RawScienceFrame.astrom.CRVAL2<dec_SA113+ddec_SA113) & (RawScienceFrame.astrom.CRVAL2>dec_SA113-ddec_SA113)
qrawsf_SA113= qrawsfall_SA113 & (RawScienceFrame.OBJECT=='STD,ZEROPOINT') & (RawScienceFrame.filter.name=='OCAM_r_SDSS') & (RawScienceFrame.EXPTIME==75.0) 

rawsf_filenames_SA113 = [rawframe.filename for rawframe in qrawsf_SA113]
redsf_filenames_SA113 = [(ReducedScienceFrame.raw.filename == rawf_filename)[0].filename for rawf_filename in rawsf_filenames_SA113]


run = 'off'

if run == 'on':
    p = Pars(AstrometricParameters)
    p.AstrometricParameters.process_params.REFCAT = 'USNO-B1.0'
    p_dict = p.get()

    dpu.run('Astrometry', instrument='OMEGACAM', pars=p_dict, red_filenames=[redsf_filenames_SA113[0]], C=1, dpu_time=1799, send_code=False)

Use the following format to create instrument specific defaults:
Pars(<class>, instrument=<instrument>, filter=<filter>, chip=<chip>)
[coma-5.vm.astro.rug.nl] 2020-05-20T12:06:49 - Calling: Processor.run(Astrometry, instrument=OMEGACAM, red_filenames=['Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58968.4195487-bb31fbd6f1b5464ea6fa4300907920ef232e4f55.fits'], pars={'AstrometricParameters.process_params.REFCAT': 'USNO-B1.0'}, C=1)
[coma-5.vm.astro.rug.nl] 2020-05-20T12:06:49 - Estimated process time : 1799 seconds
[coma-5.vm.astro.rug.nl] 2020-05-20T12:06:50 - Sending job with identifier 114943 to DPU (dpu.hpc.rug.astro-wise.org)


114943

In [152]:
dpu.get_status(114943)

[coma-5.vm.astro.rug.nl] 2020-05-20T12:11:08 - Jobid 114943 has status FINISHED N/E/A/S/U 1/0/0/0/0



#### AstrometricParameters with USNO-B1.0 as REFCAT

In [200]:
red = (ReducedScienceFrame.filename == redsf_filenames_SA113[0])
AS = (AstrometricParameters.reduced.filename == red[0].filename)

In [201]:
for As in AS:
    print(As.process_params.REFCAT)

SOURCECOLLECTIONDATA
USNO-B1.0
SA113_GAIA_patch_Willeke


<a id = "ast" SA113></a>

### 3) Using the [HOW-TO SourceLists in the Astro-WISE System](http://doc.astro-wise.org/man_howto_sourcelists.html) to obtain SourceLists

In [1]:
query = (ReducedScienceFrame.filename == redsf_filenames_SA113[0])  
frame = query[0]
astrom = ((AstrometricParameters.reduced == frame) & (AstrometricParameters.process_params.REFCAT=='USNO-B1.0')).max('creation_date')
frame.retrieve()
frame.weight.retrieve()
sourcelist = SourceList()
sourcelist.frame = frame
sourcelist.sexparam = ['MAG_APER', 'MAGERR_APER', 'FLUX_AUTO', 'FLUXERR_AUTO', 'FLUX_APER', 'FLUXERR_APER', 'ERRX2_IMAGE', 'ERRY2_IMAGE', 'ERRXY_IMAGE', 'XWIN', 'YWIN', 'ERRX2WIN', 'ERRY2WIN', 'ERRXYWIN', 'AWIN', 'BWIN', 'ERRAWIN', 'ERRBWIN', 'ERRTHETAWIN', 'THETAWIN', 'X2WIN', 'Y2WIN', 'XYWIN', 'CXXWIN', 'CYYWIN', 'CXYWIN', 'ERRCXXWIN', 'ERRCYYWIN', 'ERRCXYWIN']
sourcelist.astrom_params = astrom
sourcelist.make()
sourcelist.commit()

In [92]:
redsf_frames_SA113 = [(ReducedScienceFrame.filename == redf_filename)[0] for redf_filename in redsf_filenames_SA113]

In [153]:
run = 'off'

if run == 'on':
    astrom_113 = [((AstrometricParameters.reduced == frame) & (AstrometricParameters.process_params.REFCAT=='USNO-B1.0')).max('creation_date') for frame in redsf_frames_SA113]

    p = Pars(SourceList)
    p.SourceList.astrom_params == (AstrometricParameters.process_params.REFCAT=='USNO-B1.0')
    p_dict = p.get()

    dpu.run('SourceList', filenames=redsf_filenames_SA113, pars=p_dict, dpu_time=1799, C=1, sexparam=['MAG_APER', 'MAGERR_APER', 'FLUX_AUTO', 'FLUXERR_AUTO', 'FLUX_APER', 'FLUXERR_APER', 'ERRX2_IMAGE', 'ERRY2_IMAGE', 'ERRXY_IMAGE', 'XWIN', 'YWIN', 'ERRX2WIN', 'ERRY2WIN', 'ERRXYWIN', 'AWIN', 'BWIN', 'ERRAWIN', 'ERRBWIN', 'ERRTHETAWIN', 'THETAWIN', 'X2WIN', 'Y2WIN', 'XYWIN', 'CXXWIN', 'CYYWIN', 'CXYWIN', 'ERRCXXWIN', 'ERRCYYWIN', 'ERRCXYWIN'], send_code=False)


#### --> Creating SourceLists for given AstrometricParameters for SA113, having REFCAT "SA113_GAIA_patch_Willeke"  SLID=76167351

In [195]:
redsf_frames_SA113 = [(ReducedScienceFrame.filename == redf_filename)[0] for redf_filename in redsf_filenames_SA113]

In [2]:

run = 'off'

if run == 'on':
    astrom_113 = [((AstrometricParameters.reduced == frame) & (AstrometricParameters.process_params.REFCAT=='SA113_GAIA_patch_Willeke')).max('creation_date') for frame in redsf_frames_SA113]

    p = Pars(SourceList)
    p.SourceList.astrom_params == (AstrometricParameters.process_params.REFCAT=='SA113_GAIA_patch_Willeke')
    p_dict = p.get()

    dpu.run('SourceList', filenames=redsf_filenames_SA113, pars=p_dict, dpu_time=1799, C=1, sexparam=['MAG_APER', 'MAGERR_APER', 'FLUX_AUTO', 'FLUXERR_AUTO', 'FLUX_APER', 'FLUXERR_APER', 'ERRX2_IMAGE', 'ERRY2_IMAGE', 'ERRXY_IMAGE', 'XWIN', 'YWIN', 'ERRX2WIN', 'ERRY2WIN', 'ERRXYWIN', 'AWIN', 'BWIN', 'ERRAWIN', 'ERRBWIN', 'ERRTHETAWIN', 'THETAWIN', 'X2WIN', 'Y2WIN', 'XYWIN', 'CXXWIN', 'CYYWIN', 'CXYWIN', 'ERRCXXWIN', 'ERRCYYWIN', 'ERRCXYWIN'], send_code=False)


### 4) Using DPU to obtain SourceList calibrated to USNO for SA 113 and SA 107

In [ ]:
dpu.run('SourceList', filenames=[redsf_filenames_SA113], dpu_time=1799, C=1, astrom_params=((AstrometricParameters.reduced == frame) & (AstrometricParameters.process_params.REFCAT='USNO-B1.0')).max('creation_date'), sexparam=['MAG_APER', 'MAGERR_APER', 'FLUX_AUTO', 'FLUXERR_AUTO', 'FLUX_APER', 'FLUXERR_APER', 'ERRX2_IMAGE', 'ERRY2_IMAGE', 'ERRXY_IMAGE', 'XWIN', 'YWIN', 'ERRX2WIN', 'ERRY2WIN', 'ERRXYWIN', 'AWIN', 'BWIN', 'ERRAWIN', 'ERRBWIN', 'ERRTHETAWIN', 'THETAWIN', 'X2WIN', 'Y2WIN', 'XYWIN', 'CXXWIN', 'CYYWIN', 'CXYWIN', 'ERRCXXWIN', 'ERRCYYWIN', 'ERRCXYWIN'], send_code=False)
dpu.run('SourceList', filenames=[redsf_filenames_SA107], dpu_time=1799, C=1, astrom_params=((AstrometricParameters.reduced == frame) & (AstrometricParameters.process_params.REFCAT='USNO-B1.0')).max('creation_date'), sexparam=['MAG_APER', 'MAGERR_APER', 'FLUX_AUTO', 'FLUXERR_AUTO', 'FLUX_APER', 'FLUXERR_APER', 'ERRX2_IMAGE', 'ERRY2_IMAGE', 'ERRXY_IMAGE', 'XWIN', 'YWIN', 'ERRX2WIN', 'ERRY2WIN', 'ERRXYWIN', 'AWIN', 'BWIN', 'ERRAWIN', 'ERRBWIN', 'ERRTHETAWIN', 'THETAWIN', 'X2WIN', 'Y2WIN', 'XYWIN', 'CXXWIN', 'CYYWIN', 'CXYWIN', 'ERRCXXWIN', 'ERRCYYWIN', 'ERRCXYWIN'], send_code=False)

In [ ]:
dpu.run('SourceList', filenames=[redsf_filenames_SA113], dpu_time=1799, C=1, astrom_params=((AstrometricParameters.reduced == frame) & (AstrometricParameters.process_params.REFCAT='USNO-B1.0')).max('creation_date'), sexparam=['MAG_APER', 'MAGERR_APER', 'FLUX_AUTO', 'FLUXERR_AUTO', 'FLUX_APER', 'FLUXERR_APER', 'ERRX2_IMAGE', 'ERRY2_IMAGE', 'ERRXY_IMAGE', 'XWIN', 'YWIN', 'ERRX2WIN', 'ERRY2WIN', 'ERRXYWIN', 'AWIN', 'BWIN', 'ERRAWIN', 'ERRBWIN', 'ERRTHETAWIN', 'THETAWIN', 'X2WIN', 'Y2WIN', 'XYWIN', 'CXXWIN', 'CYYWIN', 'CXYWIN', 'ERRCXXWIN', 'ERRCYYWIN', 'ERRCXYWIN'], send_code=False)
dpu.run('SourceList', filenames=[redsf_filenames_SA107], dpu_time=1799, C=1, astrom_params=((AstrometricParameters.reduced == frame) & (AstrometricParameters.process_params.REFCAT='USNO-B1.0')).max('creation_date'), sexparam=['MAG_APER', 'MAGERR_APER', 'FLUX_AUTO', 'FLUXERR_AUTO', 'FLUX_APER', 'FLUXERR_APER', 'ERRX2_IMAGE', 'ERRY2_IMAGE', 'ERRXY_IMAGE', 'XWIN', 'YWIN', 'ERRX2WIN', 'ERRY2WIN', 'ERRXYWIN', 'AWIN', 'BWIN', 'ERRAWIN', 'ERRBWIN', 'ERRTHETAWIN', 'THETAWIN', 'X2WIN', 'Y2WIN', 'XYWIN', 'CXXWIN', 'CYYWIN', 'CXYWIN', 'ERRCXXWIN', 'ERRCYYWIN', 'ERRCXYWIN'], send_code=False)